# Set up the enviroment

In [5]:
%%capture --no-stderr
%pip install "pyautogen>=0.2.3"
%pip install -q kserve

In [6]:
# Imports
import os
import autogen

# Define your model

In [ ]:
from kserve import KServeClient

KServe = KServeClient()
namespace = "my-profile"
isvc_resp = KServe.get("llama2", namespace=namespace)
isvc_url = isvc_resp["status"]["address"]["url"]

print(f"Make an API call to: {isvc_url}")

In [7]:
# Specify the model name and the endpoint
MODEL_NAME = "llama2"
LB_ENDPOINT = "localhost:8000"
os.environ["FORCED_API_KEY"] = "null"
os.environ["AUTOGEN_USE_DOCKER"] = "0"

In [8]:
# load the config list from enviroment variables
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='.env',
    model_api_key_map={
        MODEL_NAME: {
            "api_key_env_var": "FORCED_API_KEY",
            "base_url": "http://"+ LB_ENDPOINT + "/v2"
        }
    },
    filter_dict={
        "model": {
           MODEL_NAME,
        }
    }
)

config_list

INFO:root:Models available: ['llama2']


[{'api_key': 'null',
  'base_url': 'http://localhost:8000/v2',
  'model': 'llama2'}]

# Configure your agents


In [9]:
gpt_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0, # set the forced prompt temp to 0
    "config_list": config_list, # load the config from above
    "timeout": 120, # set timeout to 120 seconds
}

user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
''',
)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code."""
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
''',
    llm_config=gpt_config,
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt_config,
)

groupchat = autogen.GroupChat(agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt_config)

# Begin Group Chat


In [10]:
user_proxy.initiate_chat(
    manager,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------
